# Journal


## 1. Decide What To Do First When Approaching My First Full-Stacked Software Development
I am not a professional software developer. My background is primarily in data science, and before this project, the only languages I was comfortable with were Python, SQL, and R. I had little to no experience with frontend development technologies such as HTML, JavaScript, and Typescript, nor did I fully understand how API worked or how frontend and backend communicate with each other.

Despite these gaps, the development process moved much faster than I had expected due to the assistance of multiple AI tools. After three years of working closely with our property manager, I had developed a strong understanding of real-world property management operations, including pain points, workflows, and inefficiencies. I began by writing a high-level project overview that clarified the core problems I wanted to solve and the key system components required. Instead of starting with backend logic, I chose to begin by creating the frontend prototype using **Google AI Studio**. This allowed me to visualize the system early on and quickly iterate on ideas. Seeing the interface in action helped translate abstract requirements into concrete features and workflows.

Once the frontend direction became clearer, I moved on to designing the database schema using **Lucidchart**. This step helped me formalize relationships between entities and ensured that the system structure could support the real operational needs I had observed in practice. Although as I started to develop the database schema and backend logic, I needed to rework my frontend. Starting somewhere still was a valuable experience and an import step.



## 2. Decide My Technology Stack

## Where To Deploy Postgres Database
My final goal is to deploy my Postgres database on a Cloud Platform like AWS RDS. RDS has a free-tier option available for early-stage development. However, I decided to develop on my own local machine using a Docker container to avoid all the messy configuration and authentication issues. Because my early stage involves many reset steps to the database, having a local database will make the development process a lot quicker.



## 3. Determining Primary Keys for Room-Related Operation Tables

### Background
In my business domain, a room is naturally and uniquely identified by the composite key:

- `building_no`
- `floor_no`
- `room_no`

For example:
- Building 2 – Floor 4 – Room C
- Building 4 – Floor 4 – Room C

Although these rooms share the same floor and room number, they are different business entities.

When designing **room-related operation tables** (e.g. `invoice`, `payment`, `electricity_bill`), a conflict emerged between **business logic** and **database design best practices**.

---

### The Design Conflict
Since invoices and payments are associated with a room, two design choices presented themselves.

| Option | Description | Pros | Cons |
|------|-------------|------|------|
| **Option 1: Surrogate Key (`room_id`)** | Use an artificial primary key for `room` and reference it from all room-related operation tables (e.g. `invoice`, `payment`). | - Fully normalized<br>- No duplicated data<br>- Compact foreign keys<br>- Efficient joins and indexes<br>- ORM-friendly<br>- Safe for future changes | - Requires an extra lookup step to translate `(building_no, floor_no, room_no)` into `room_id`<br>- Surrogate key has no direct business meaning, which initially feels awkward |
| **Option 2: Composite Natural Key** | Use `(building_no, floor_no, room_no)` as the primary key and propagate it to all related tables. | - No lookup step required<br>- Keys align directly with business concepts | - Every foreign key becomes 3 columns<br>- Index size and complexity grow quickly<br>- ORM & migrations become painful<br>- Refactoring is risky (room renumbering, building merges)<br>- Poor long-term scalability |

---

### The chosen Solution
The correct solution is a hybrid approach:
1. Keep using a surrogate key (room_id) as the primary key
2. Enforce the natural business key with a unique constraint

```sql
UNIQUE (building_no, floor_no, room_no)
```

3. Translate business identity to surrogate key at system boundaries

---

### Practical Implementation
1. Room Lookup View
Create a view to bridge human-readable identity and internal IDs:
```sql
CREATE VIEW v_room_lookup AS
SELECT
  room_id,
  building_no,
  floor_no,
  room_no,
  building_no || '-' || floor_no || '-' || room_no AS room_label
FROM room;
```
This supports dashboards, debugging, and UI display without leaking composite keys into operation tables.

2. Backend Helper Function
Wrap the lookup logic in a single reusable function:
```python
def get_room_id(building_no, floor_no, room_no):
    return db.fetchval("""
        SELECT room_id
        FROM room
        WHERE building_no = %s
          AND floor_no = %s
          AND room_no = %s
    """, building_no, floor_no, room_no)

```

---

### Key Takeway
- Use surrogate keys for relationships.
- Enforce natural keys for correctness.
- Translate between them at system boundaries.


The initial discomfort of an extra lookup is a small, controlled cost that prevents significant architectural debt in the future.

## 4. Whether To Store Lease Status In A Table

Status is a derived business state. It is not a primary fact so it should be computed, not manually edited. Some exceptions are:
- If there are millions of leases, and computing the status took tons of time
- If our system needs ultra-fast filtering on the status
- If we are willing to maintain triggers


## 5. Whether To Implement A Lease Modification Functionality
A key design question is how to handle changes after a lease is created without compromising accounting integrity or auditability , especially when rent increases, data-entry mistakes and room swaps happen.

Think of the question: When the rent is raised, should the original leases be modified directly, or should they be terminated first and then a new one be issued? If we decided to terminate the original contract to maintain data integrity, what if, on one day, the property manager does an incorrect data entry and realizes only after the contract is active? Should we terminate the contract as well? This introduces the importance of lease status and lease amendment.

Core Principle<br>
A lease becomes immutable once it is financially effective. Any change after that point must be recorded as a new fact, not a mutation of history.

A lease is editable only if all of the following conditions are met:

- Lease status is DRAFT or PENDING
- No invoices have been generated
- No payments have been recorded 

That way, we can maintain accounting integrity, audit safety, and prevent retroactive financial corruption. Once a lease is active, it should become immutable, and all the changes should be recorded into an amendment table.

The description of different statuses and their decision rules is displayed below.
- `DRAFT`: created, not finalized, fully editable
- `PENDING`: signed/awaiting start date, still editable
- `ACTIVE`: in effect, locked for editing
- `TERMINATED`: ended early
- `EXPIRED`: ended naturally

This logic only lives in the backend.
```text
IF termination_date IS NOT NULL
 → TERMINATED
ELSE IF today < start_date
 → PENDING
ELSE IF today BETWEEN start_date AND end_date
 → ACTIVE
ELSE
 → EXPIRED
```

The amendment table handles lease modification after it is active, for instance, when the landlord raises the rent. The amendments never overwrite history, are time-bound, and auditable. Financial calculations reference lease base rent plus applicable amendments by period. This cleanly supports
- Rent increases
- Discounts/promotions
- Negotiated changes
- Legal audit trials

However, when two tenants swap rooms, we should create new contracts because a lease amendment modifies the terms of a lease, not the subject of a lease. Therefore, `room_id` is not included in the lease_amendment table. 



```sql
CREATE TABLE lease (
    id BIGINT GENERATED ALWAYS AS IDENTITY,
    room_id BIGINT NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE NOT NULL,
    early_termination_date DATE,

    termination_reason TEXT, -- added later

    monthly_rent NUMERIC(10,2) NOT NULL,
    deposit NUMERIC(10,2) NOT NULL,
    pay_rent_on SMALLINT NOT NULL,
    payment_term payment_term_type NOT NULL,
    status lease_status NOT NULL,
    assets JSONB,
    vehicle_plate TEXT,
    created_by BIGINT,
    updated_by BIGINT,
    created_at TIMESTAMPTZ NOT NULL DEFAULT now(),
    updated_at TIMESTAMPTZ,
    deleted_at TIMESTAMPTZ,

    CONSTRAINT pk_lease PRIMARY KEY (id),
    CONSTRAINT fk_lease_room
        FOREIGN KEY (room_id) REFERENCES room(id),
    CONSTRAINT chk_lease_dates CHECK (end_date > start_date),
    CONSTRAINT chk_early_termination
        CHECK (
            early_termination_date IS NULL
            OR early_termination_date BETWEEN start_date AND end_date
        ),
    CONSTRAINT chk_monthly_rent CHECK (monthly_rent >= 0),
    CONSTRAINT chk_deposit CHECK (deposit >= 0),
    CONSTRAINT chk_pay_rent_on CHECK (pay_rent_on BETWEEN 1 AND 31),
    CONSTRAINT fk_lease_created_by
        FOREIGN KEY (created_by) REFERENCES user_account(id),
    CONSTRAINT fk_lease_updated_by
        FOREIGN KEY (updated_by) REFERENCES user_account(id)    
);

-- Create a whole new table lease_amendment
CREATE TYPE lease_amendment_type AS ENUM (
    'RENT_CHANGE',
    'DISCOUNT',
    'OTHER'
);

CREATE TABLE lease_amendment (
    id BIGINT GENERATED ALWAYS AS IDENTITY,
    lease_id BIGINT NOT NULL,

    amendment_type lease_amendment_type NOT NULL,

    effective_date DATE NOT NULL,

    -- Financial deltas (nullable depending on amendment_type)
    old_monthly_rent NUMERIC(10,2),
    new_monthly_rent NUMERIC(10,2),

    reason TEXT NOT NULL,
    discount_type TEXT CHECK (
        discount_type IN ('FREE_MONTHS', 'FIXED_AMOUNT', 'PERCENTAGE')
    ),
    created_by BIGINT NOT NULL,
    created_at TIMESTAMPTZ NOT NULL DEFAULT now(),
    deleted_at TIMESTAMPTZ,

    CONSTRAINT pk_lease_amendment PRIMARY KEY (id),

    CONSTRAINT fk_lease_amendment_lease
        FOREIGN KEY (lease_id) REFERENCES lease(id),

    CONSTRAINT fk_lease_amendment_created_by
        FOREIGN KEY (created_by) REFERENCES user_account(id),

    -- Ensure effective date is sane (cannot precede lease start)
    CONSTRAINT chk_amendment_effective_date
        CHECK (effective_date >= (
            SELECT start_date FROM lease WHERE lease.id = lease_id
        )),

    -- Rent change amendments must have both values
    CONSTRAINT chk_rent_change_values
        CHECK (
            amendment_type <> 'RENT_CHANGE'
            OR (
                old_monthly_rent IS NOT NULL
                AND new_monthly_rent IS NOT NULL
                AND old_monthly_rent >= 0
                AND new_monthly_rent >= 0
            )
        )
);

```


    


1. Fill Database for
    - tenant
    - lease
    - lease_amendment
    - lease_tenant
    - invoice








# 6. Divide The Whole Project By Phase And By Small Steps
Vibe coding only makes the developing processes more effective if the developer knows how to structure the prompts well enough. This means how to set boundaries of each services. Otherwise, AI may generate codes that mixed up the funcitons or keep appending codes to current code base. 

### 1. Tenant Set Up and Contract Assigment
